In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Data Pipeline & High-Level Workflow

In previous tutorials, we manually iterated the solver step-by-step (`solver.step()`). 
For production runs, analysis, or parameter sweeps, it is more efficient to use the **High-Level API** (`solver.solve()`) along with **Callbacks**.

This tutorial demonstrates:
1.  **Robust Configuration**: Setting up a stable simulation for Earth-scale physics.
2.  **Mock Data Ingestion**: How one might map external data to the solver.
3.  **The `solve()` API**: Running a simulation with automated time-stepping and monitoring.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereSWE, SWEConfig

## 1. Robust Configuration

Stability on the Cubed-Sphere is governed by the CFL condition, which scales with $\frac{1}{N^2}$.
For global simulations (Earth Radius $R \approx 6371$ km), the time step ($dt$) must be carefully chosen based on the maximum wave speed.

In [ ]:
# Physical Constants (Earth)
R_earth = 6.37122e6  # Meters
g = 9.80616          # m/s^2
H_avg = 10000.0      # Average depth (10km)
N = 16               # Polynomial Order (Resolution)

# --- Dynamic Stability Calculation ---
# 1. Gravity Wave Speed: c = sqrt(gH)
c_wave = np.sqrt(g * H_avg)

# 2. Max Flow Velocity (Estimate for standard zonal flows)
u_flow_max = 40.0 # m/s (approx jet stream)

# 3. Total Characteristic Speed
v_max_total = c_wave + u_flow_max

# 4. Stable Time Step (CFL Heuristic for DG)
# dt ~ CFL * R / (v_max * N^2)
target_cfl = 0.5
dt_safe = target_cfl * R_earth / (v_max_total * N**2)

print(f"Gravity Wave Speed: {c_wave:.1f} m/s")
print(f"Max Total Speed:    {v_max_total:.1f} m/s")
print(f"Calculated Safe dt: {dt_safe:.2f} s")

# Initialize Config with Calculated dt
config = SWEConfig(
    N=N, 
    R=R_earth, 
    gravity=g, 
    H_avg=H_avg,
    backend='numpy', # Standard backend
    dt=dt_safe       # Explicit stable step
)

solver = CubedSphereSWE(config)

## 2. Data Ingestion (Mock Scenario)

In a real pipeline, you would load weather data (e.g., GFS, ECMWF) on a Lat/Lon grid and interpolate it to the Cubed-Sphere faces.

Here, we simulate this process by "loading" the **Williamson Case 2** initial condition.

In [ ]:
# Mock: Create a source Lat/Lon grid (Conceptual)
lat_source = np.linspace(-90, 90, 180)
lon_source = np.linspace(0, 360, 360)
LAT, LON = np.meshgrid(lat_source, lon_source)
# data_source = load_from_netcdf(...) 

# For this tutorial, we generate the clean Case 2 state directly
# This represents our "Ingested Initial Condition"
initial_state = solver.get_initial_condition(type="case2")

print(f"Initial State Shape: {initial_state.shape} (Vars, Face, Xi, Eta)")
# Vars: 0=Mass(h*sqrt_g), 1=Momentum_1, 2=Momentum_2

## 3. High-Level Execution: `solver.solve()`

Instead of writing a manual `for` loop, we use `solve()`. This method handles the time-integration loop and accepts **Callbacks** for monitoring or I/O.

In [ ]:
# Define a simple Monitor Callback
def simple_monitor(t, state):
    # This function runs periodically during the solver loop
    # We can calculate metrics like Total Mass to check conservation
    
    # State[0] is Mass variable (h * sqrt_g)
    # We should technically sum(state[0] * w_alpha * w_beta), 
    # but simple summation is a fast approximated proxy for checking NaNs/Explosions.
    current_mass_proxy = np.sum(state[0])
    
    # Print status
    # \r allows updating the same line in some consoles (optional)
    print(f"[Monitor] Time: {t:.1f}s | Mass Proxy: {current_mass_proxy:.4e}")

# Define Simulation Span (e.g., 1 Hour)
t_hours = 1.0
t_final = t_hours * 3600.0
t_span = (0.0, t_final)

print(f"Starting Simulation for {t_hours} hour(s)...")

# Execute Solver
# We pass our custom callback in a list
# The solver uses the 'dt' from config or the one passed here
final_state = solver.solve(t_span, initial_state, callbacks=[simple_monitor])

print("Simulation Complete.")

## 4. Post-Processing and Visualization

With the simulation complete, we visualize the final Geopotential Height ($h$).

In [ ]:
# Extract Height Field
# The state variable 0 is Mass = h * sqrt_g
# To get h, we divide by sqrt_g
# Let's visualize Face 0 (Equatorial)

face_idx = 0
face_name = list(solver._impl.topology.FACE_MAP)[face_idx]
sqrt_g = solver._impl.faces[face_name].sqrt_g

h_field = final_state[0, face_idx] / sqrt_g

plt.figure(figsize=(10, 8))
plt.imshow(h_field, origin='lower', extent=[-45, 45, -45, 45], cmap='viridis')
plt.colorbar(label='Geopotential Height (m)')
plt.title(f"Final Height Field (Face {face_idx}) @ T={t_final/3600:.1f}h")
plt.xlabel('Xi (iso-parametric)')
plt.ylabel('Eta (iso-parametric)')
plt.show()